In [1]:
import torch
import numpy as np
import pickle
import os
import sys

In [2]:
activations = torch.load('data/activations_train.pth', weights_only=True)
activations.shape

torch.Size([16214, 512])

In [6]:
with open('data/concepts.pkl', 'rb') as f:
    concepts = pickle.load(f)

concepts

['5_o_Clock_Shadow',
 'Arched_Eyebrows',
 'Attractive',
 'Bags_Under_Eyes',
 'Bald',
 'Bangs',
 'Big_Lips',
 'Big_Nose',
 'Black_Hair',
 'Blond_Hair',
 'Blurry',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'Chubby',
 'Double_Chin',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Heavy_Makeup',
 'High_Cheekbones',
 'Male',
 'Mouth_Slightly_Open',
 'Mustache',
 'Narrow_Eyes',
 'No_Beard',
 'Oval_Face',
 'Pale_Skin',
 'Pointy_Nose',
 'Receding_Hairline',
 'Rosy_Cheeks',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie',
 'Young']

In [7]:
concepts = [
 "5_o_Clock_Shadow",
 "Arched_Eyebrows",
 "Attractive",
 "Bags_Under_Eyes",
 "Bald",
 "Bangs",
 "Big_Lips",
 "Big_Nose",
 "Black_Hair",
 "Blond_Hair",
 "Blurry",
 "Brown_Hair",
 "Bushy_Eyebrows",
 "Chubby",
 "Double_Chin",
 "Eyeglasses",
 "Goatee",
 "Gray_Hair",
 "Heavy_Makeup",
 "High_Cheekbones",
 "Male",
 "Mouth_Slightly_Open",
 "Mustache",
 "Narrow_Eyes",
 "No_Beard",
 "Oval_Face",
 "Pale_Skin",
 "Pointy_Nose",
 "Receding_Hairline",
 "Rosy_Cheeks",
 "Sideburns",
 "Smiling",
 "Straight_Hair",
 "Wavy_Hair",
 "Wearing_Earrings",
 "Wearing_Hat",
 "Wearing_Lipstick",
 "Wearing_Necklace",
 "Wearing_Necktie",
 "Young",
 "timestamp",
 "box"
]

with open('data/concept_names.pkl', 'wb') as f:
    pickle.dump(concepts, f)